<h1><center>LangChain 101: from Zero to Hero</center></h1>
<hr><hr><hr>

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

True

### Configurations:-
----------------------

In [3]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_api_version = "2023-05-15"
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")

os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = azure_openai_api_version
os.environ["OPENAI_API_KEY"]      = azure_openai_api_key

## 1. The LLMs:
----------------
The fundamental component of LangChain involves invoking an LLM with a specific input. <br>
To illustrate this, we'll explore a simple example. Let's imagine we are building a service that *suggests personalized workout routines based on an individual's fitness goals and preferences*.

To accomplish this, we will first need to import the LLM wrapper.

In [4]:
from langchain_openai import AzureChatOpenAI

- The `temperature` parameter in OpenAI models manages the randomness of the output.
- When `temperature` is set to 0, the output is mostly predetermined and suitable for tasks requiring stability and the most probable result. At a setting of 1.0, the output can be inconsistent and interesting but isn't generally advised for most tasks.
- For creative tasks, a `temperature` between **0.70 and 0.90** offers a balance of reliability and creativity. The best setting should be determined by experimenting with different values for each specific use case.

In [5]:
llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0.9
)

Calling the model with some input- This code will generate a personalized workout routine based on the user's fitness goals and preferences using the LLM model:

In [8]:
text = "Suggest a personalized workout routine for someone looking to improve cardiovascular endurance and prefers outdoor activities."

response = llm.invoke( text )
print( response.content )

Sure! Here's a personalized workout routine that focuses on improving cardiovascular endurance while incorporating outdoor activities:

1. Warm-up (5-10 minutes):
   - Start with a light jog or brisk walk to warm up your muscles.
   - Perform dynamic stretches like leg swings, arm circles, and lunges.

2. Outdoor Running (3 times a week):
   - Choose a scenic route or park with varied terrain.
   - Start with a comfortable pace and gradually increase your speed over time.
   - Incorporate interval training by alternating between a moderate pace and a faster, more challenging pace for specific time intervals (e.g., 1 minute fast, 2 minutes moderate).
   - Aim for a minimum of 30 minutes of continuous running per session, gradually increasing the duration as you progress.

3. Cycling (2 times a week):
   - Invest in a bicycle or rent one from a local rental service.
   - Explore different cycling routes, such as bike trails or scenic roads.
   - Start with moderate cycling speed and grad

## 2. The Chains:
------------------
In LangChain, a chain is an end-to-end wrapper around multiple individual components, providing a way to accomplish a common use case by combining these components in a specific sequence. The most commonly used type of chain is the LLMChain, which consists of a PromptTemplate, a model (either an LLM or a ChatModel), and an optional output parser.

The LLMChain works as follows:
1. Takes (multiple) input variables.
2. Uses the PromptTemplate to format the input variables into a prompt.
3. Passes the formatted prompt to the model (LLM or ChatModel).
4. If an output parser is provided, it uses the OutputParser to parse the output of the LLM into a final format.

Below is the code to create a chain that generates a possible name for a company that produces eco-friendly water bottles. By using LangChain's LLMChain, PromptTemplate, and OpenAIclasses, we can easily define our prompt, set the input variables, and generate creative outputs. 

In [6]:
from langchain.prompts import PromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.chains import LLMChain

In [7]:
llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0.9
)

prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)


llm_chain = LLMChain( prompt=prompt, llm=llm )

In [13]:

# Run the chain only specifying the input variable.
response = llm_chain.invoke("eco-friendly water bottles")
print( response )

{'product': 'eco-friendly water bottles', 'text': 'EcoAqua'}


In [15]:
print( response["text"] )

EcoAqua


The same chain can also be built as follow:-
---------------------------------------------

In [16]:
from langchain.prompts import ChatPromptTemplate

In [17]:
company_name_prompt = ChatPromptTemplate.from_template("""
What is a good name for a company that makes {product}?
""")

company_name_chain = company_name_prompt | llm

In [18]:
product = "Organic Preservative-free ayurvedic supplements"

# Invoking the chain to get response from llm
response = company_name_chain.invoke( {"product": product} )

In [19]:
print( response )

content="Nature's Ayurblends"


In [20]:
print( response.content )

Nature's Ayurblends


## 3. The Memory:-
-----------------------
- In LangChain, Memory refers to the mechanism that stores and manages the conversation history between a user and the AI.
- It helps maintain context and coherency throughout the interaction, enabling the AI to generate more relevant and accurate responses.
- Memory, such as `ConversationBufferMemory`, acts as a wrapper around `ChatMessageHistory`, extracting the messages and providing them to the chain for better context-aware generation.

In [6]:
from langchain_openai import AzureChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [7]:
llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0
)

conversation = ConversationChain(
    llm=llm,
    verbose=True,
    memory=ConversationBufferMemory()
)

In [25]:
# Start the conversation
conversation.predict(input="Tell me about yourself.")

# Continue the conversation
conversation.predict(input="What can you do?")
conversation.predict(input="How can you help me with data analysis?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Tell me about yourself.
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Tell me about yourself.
AI: I'm an AI language model developed by OpenAI called GPT-3. I have been trained on a wide range of data sources, including books, articles, and websites, to generate human-like responses. I can understand and generate text in multiple languages, and I can assist wi

"I can assist with data analysis in a few different ways. For example, if you have a dataset and you're not sure how to approach analyzing it, I can help you brainstorm different approaches and techniques. I can also help you with specific tasks, such as cleaning and preprocessing data, performing statistical analysis, or creating visualizations to better understand your data. Additionally, if you have any questions about data analysis concepts or methods, I can provide explanations and examples to help clarify things for you. Just let me know what specific help you need, and I'll do my best to assist you!"

In [26]:
# Display the conversation
print(conversation)

memory=ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Tell me about yourself.'), AIMessage(content="I'm an AI language model developed by OpenAI called GPT-3. I have been trained on a wide range of data sources, including books, articles, and websites, to generate human-like responses. I can understand and generate text in multiple languages, and I can assist with a variety of tasks, such as answering questions, providing explanations, and even engaging in creative writing. Is there anything specific you would like to know or discuss?"), HumanMessage(content='What can you do?'), AIMessage(content="I can do a lot of things! As an AI language model, I can assist with various tasks. For example, I can help answer questions, provide explanations, generate text, and even engage in creative writing. If you have any specific requests or topics you'd like to discuss, feel free to let me know!"), HumanMessage(content='How can you help me with data analys

Points to note:-
---------------------
1. In the above output, you can see the memory being used by observing the "Current conversation" section.
2. After each input from the user, the conversation is updated with both the user's input and the AI's response.
3. This way, the memory maintains a record of the entire conversation.
4. When the AI generates its next response, it will use this conversation history as context, making its responses more coherent and relevant.

---------------------------------------------------------------------------------------------------------------------------

# Deep Lake VectorStore:
---------------------------
Deep Lake provides storage for embeddings and their corresponding metadata in the context of LLM apps. It enables hybrid searches on these embeddings and their attributes for efficient data retrieval. It also integrates with LangChain, facilitating the development and deployment of applications.

Deep Lake provides several advantages over the typical vector store:

- **It’s multimodal**, which means that it can be used to store items of diverse modalities, such as texts, images, audio, and video, along with their vector representations. 
- **It’s serverless**, which means that we can create and manage cloud datasets without creating and managing a database instance. This aspect gives a great speedup to new projects.
- Last, it’s possible to **easily create a data loader** out of the data loaded into a Deep Lake dataset. It is convenient for fine-tuning machine learning models using common frameworks like *PyTorch* and *TensorFlow*.

Install deeplake package:
- `pip install deeplake`

In [7]:
pip show deeplake

Name: deeplake
Version: 3.8.22
Summary: Activeloop Deep Lake
Home-page: 
Author: activeloop.ai
Author-email: support@activeloop.ai
License: MPL-2.0
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: boto3, click, humbug, lz4, numpy, pathos, pillow, pydantic, pyjwt, tqdm
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [12]:
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")
embedding_model_deployment_name = os.getenv("AZURE_OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
activeloop_token = os.getenv("ACTIVELOOP_DEEPLAKE_KEY")

os.environ["ACTIVELOOP_TOKEN"] = activeloop_token

In [13]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from langchain.vectorstores import DeepLake
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

In [10]:
llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embedding_model_deployment_name,
    openai_api_version=azure_openai_api_version,
)

In [11]:
# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

### Create deeplake dataset:
-----------------------------

In [18]:
# create Deep Lake dataset
# TODO: use your organization id here. (by default, org id is your username)
# my_activeloop_org_id = "<YOUR-ACTIVELOOP-ORG-ID>" 
my_activeloop_org_id = "debanjanbusy" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
db = DeepLake(dataset_path=dataset_path, embedding=embeddings)

Your Deep Lake dataset has been successfully created!


In [19]:
db

### Delete a deeplake dataset:
------------------------------

In [8]:
import deeplake

In [14]:
my_activeloop_org_id = "debanjanbusy" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

In [17]:
deeplake.delete( dataset_path )

### Add documents to our deeplake dataset:
-------------------------------------------

In [20]:
# create our documents
texts = [
    "Napoleon Bonaparte was born in 15 August 1769",
    "Louis XIV was born in 5 September 1638"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [21]:
# add documents to our Deep Lake dataset
response = db.add_documents(docs)

Creating 2 embeddings in 1 batches of size 2:: 100%|█████████████████████████████████████| 1/1 [00:21<00:00, 21.47s/it]

Dataset(path='hub://debanjanbusy/langchain_course_from_zero_to_hero', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (2, 1)      str     None   
 metadata     json      (2, 1)      str     None   
 embedding  embedding  (2, 1536)  float32   None   
    id        text      (2, 1)      str     None   


- The output obtained executing the code in the above cell looks somewhat like this: <br>
<img src="./images/01-output of adding documents to dataset.jpg" width="1000px" />

In [22]:
print( response )

['7bdcfde6-d6d5-11ee-8dd0-80913334ccee', '7bdcfde7-d6d5-11ee-8351-80913334ccee']


## Visualizing the Dataset using `dataset.visualize()`:
--------------------------------------------------------
- `visualize()` method requires Python `flask` module to visualize the dataset.

In [16]:
pip show flask

Name: Flask
Version: 3.0.2
Summary: A simple framework for building complex web applications.
Home-page: 
Author: 
Author-email: 
License: 
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: blinker, click, itsdangerous, Jinja2, Werkzeug
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
import deeplake

In [13]:
my_dataset = deeplake.load("hub://debanjanbusy/langchain_course_from_zero_to_hero")

\

This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/debanjanbusy/langchain_course_from_zero_to_hero



hub://debanjanbusy/langchain_course_from_zero_to_hero loaded successfully.



In [17]:
my_dataset.visualize()

HINT: Please forward the port - 50455 to your local machine, if you are running on the cloud.


 * Serving Flask app 'dataset_visualizer'
 * Debug mode: off


## Creating a `RetrievalQA` chain:
------------------------------------

In [10]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

In [14]:
llm = AzureChatOpenAI(
    openai_api_version = azure_openai_api_version,
    azure_deployment = llm_deployment_name,
    temperature = 0
)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=embedding_model_deployment_name,
    openai_api_version=azure_openai_api_version,
)

In [15]:
my_activeloop_org_id = "debanjanbusy" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# Loading an existing non-empty dataset.
db = DeepLake(dataset_path=dataset_path, read_only=True, embedding=embeddings)

Deep Lake Dataset in hub://debanjanbusy/langchain_course_from_zero_to_hero already exists, loading from the storage


In [16]:
print( db )

In [17]:
retrieval_qa = RetrievalQA.from_chain_type(
	llm=llm,
	chain_type="stuff",
	retriever=db.as_retriever()
)

#### Creating an agent that uses the above `RetrievalQA` chain as a tool:

In [24]:
from langchain.agents import initialize_agent, Tool, AgentType

In [25]:
tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.invoke,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

"""
LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
"""

'\nLangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.\n'

- Using the agent to ask a question:

In [26]:
response = agent.invoke("When was Napoleone born?")
print(response)



> Entering new AgentExecutor chain...
I should use the Retrieval QA System to find the answer to this question.
Action: Retrieval QA System
Action Input: "When was Napoleone born?"
Observation: {'query': 'When was Napoleone born?', 'result': 'Napoleon Bonaparte was born on 15 August 1769.'}
Thought:The answer to the question "When was Napoleone born?" is "Napoleon Bonaparte was born on 15 August 1769."
Final Answer: Napoleon Bonaparte was born on 15 August 1769.

> Finished chain.
{'input': 'When was Napoleone born?', 'output': 'Napoleon Bonaparte was born on 15 August 1769.'}


In [28]:
print( response["output"] )

Napoleon Bonaparte was born on 15 August 1769.


- The example above how to use Deep Lake as a vector database and create an agent with a RetrievalQA chain as a tool to answer questions based on the given document.

<hr>

## Reloading an existing vector store/dataset and adding more data to it:
--------------------------------------------------------------------------
- We first reload an existing vector store from Deep Lake that's located at a specified dataset path.
- Then, we load new textual data and split it into manageable chunks.
- Finally, we add these chunks to the existing dataset, creating and storing corresponding embeddings for each added text segment.

In [29]:
my_activeloop_org_id = "debanjanbusy" 
my_activeloop_dataset_name = "langchain_course_from_zero_to_hero"
dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"

# load the existing Deep Lake dataset and specify the embedding function
db = DeepLake(dataset_path=dataset_path, embedding=embeddings)

Using embedding function is deprecated and will be removed in the future. Please use embedding instead.


Deep Lake Dataset in hub://debanjanbusy/langchain_course_from_zero_to_hero already exists, loading from the storage


In [30]:
# create new documents
texts = [
    "Lady Gaga was born in 28 March 1986",
    "Michael Jeffrey Jordan was born in 17 February 1963"
]
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.create_documents(texts)

In [31]:
# add documents to our Deep Lake dataset
db.add_documents(docs)

Creating 2 embeddings in 1 batches of size 2:: 100%|█████████████████████████████████████| 1/1 [00:29<00:00, 29.08s/it]

Dataset(path='hub://debanjanbusy/langchain_course_from_zero_to_hero', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
 embedding  embedding  (4, 1536)  float32   None   
    id        text      (4, 1)      str     None   
 metadata     json      (4, 1)      str     None   
   text       text      (4, 1)      str     None   


['c0df26a0-d712-11ee-a9be-80913334ccef',
 'c0df26a1-d712-11ee-bbc9-80913334ccef']

- Shape = (4,1536) for embedding means there are 4 embeddings (one per document), and each embedding has 1536 columns, i.e., 1536 elementary data(float) in each row.
- Shape = (4,1) for text means there are 4 texts / documents, and each document has 1 column, i.e., single elementary data and not an array or collection.

#### We then recreate our previous agent and ask a question that can be answered only by the last documents added:

In [37]:
from langchain.agents import initialize_agent, Tool, AgentType

tools = [
    Tool(
        name="Retrieval QA System",
        func=retrieval_qa.invoke,
        description="Useful for answering questions."
    ),
]

agent = initialize_agent(
	tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

"""
LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
"""

'\nLangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.\n'

In [39]:
response = agent.invoke("When was Michael Jeffrey Jordan born?")
print(response)



> Entering new AgentExecutor chain...
I should use the Retrieval QA System to find the answer to this question.
Action: Retrieval QA System
Action Input: "When was Michael Jeffrey Jordan born?"
Observation: {'query': 'When was Michael Jeffrey Jordan born?', 'result': 'Michael Jeffrey Jordan was born on February 17, 1963.'}
Thought:I now know the final answer
Final Answer: Michael Jeffrey Jordan was born on February 17, 1963.

> Finished chain.
{'input': 'When was Michael Jeffrey Jordan born?', 'output': 'Michael Jeffrey Jordan was born on February 17, 1963.'}


-----------------------------------------------------

## Agents in LangChain:
------------------------
In LangChain, agents are high-level components that use language models (LLMs) to determine which actions to take and in what order. An action can either be using a tool and observing its output or returning it to the user. Tools are functions that perform specific duties, such as Google Search, database lookups, or Python REPL.

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.

Several types of agents are available in LangChain:

- The `zero-shot-react-description` agent uses the ReAct framework to decide which tool to employ based purely on the tool's description. It necessitates a description of each tool.
- The `react-docstore` agent engages with a docstore through the ReAct framework. It needs two tools: a Search tool and a Lookup tool. The Search tool finds a document, and the Lookup tool searches for a term in the most recently discovered document.
- The `self-ask-with-search` agent employs a single tool named Intermediate Answer, which is capable of looking up factual responses to queries. It is identical to the original self-ask with the search paper, where a Google search API was provided as the tool.
- The `conversational-react-description` agent is designed for conversational situations. It uses the ReAct framework to select a tool and uses memory to remember past conversation interactions.

In the below example, the Agent will use the Google Search tool to look up recent information about the Mars rover and generates a response based on this information.

### Building Agent using `PythonREPL` Tool:-
----------------------------------------------
- The `PythonREPL` tool is contained within the `langchain_experimental` module, so it needs to be installed first.
- Sometimes, for complex calculations, rather than have an LLM generate the answer directly, it can be better to have the LLM generate code to calculate the answer, and then run that code to get the answer. In order to easily do that, we provide a simple Python REPL to execute commands in.

- This interface will only return things that are printed - therefore, **if you want to use it to calculate an answer, make sure to have it print out the answer.**

In [45]:
pip show langchain-experimental

Name: langchain-experimental
Version: 0.0.53
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: e:\programs & codes\generative ai\_genai_venv\lib\site-packages
Requires: langchain, langchain-core
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [46]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

In [47]:
python_repl = PythonREPL()

In [53]:
response = python_repl.run("print(1+1)")
print( response )

2



In [61]:
sum_code = """
sum = 0
for i in range(1,11):
    sum += i
print(sum)
"""

response = python_repl.run( sum_code )
print( response )

55



In [64]:
# del repl_tools
# del python_repl_agent

In [63]:
# You can create the tool to pass to an agent
repl_tools = [
    Tool(
        name="python_repl",
        description="A Python interpreter. Use this to execute python code. Input should be a valid python code. If you want to see the output of a value, you should print it out with `print(...)`.",
        func=python_repl.run,
    ),
]

python_repl_agent = initialize_agent(
	repl_tools,
	llm,
	agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
	verbose=True
)

In [65]:
n = int(input("Enter a number whose factorial is to be calculated: "))

factorial_code = f"""
fact = 1
for i in range(1,{n+1}):
    fact *= i
print( fact )
"""

response = python_repl_agent.invoke( factorial_code )
print( response )

Enter a number whose factorial is to be calculated:  5




> Entering new AgentExecutor chain...
The code calculates the factorial of 5. I can run the code to see the result.

Action: python_repl
Action Input:
fact = 1
for i in range(1,6):
    fact *= i
print( fact )

Observation: 120

Thought:The code correctly calculates the factorial of 5, which is 120.
Final Answer: 120

> Finished chain.
{'input': '\nfact = 1\nfor i in range(1,6):\n    fact *= i\nprint( fact )\n', 'output': '120'}


In [66]:
print( response["output"] )

120
